In [1]:
import cv2
import mediapipe as mp

import ctypes
import os
import sys
import traceback
import shutil

In [2]:
print(os.getcwd())

c:\Users\ivano\OneDrive\Документы\GitHub\Gloss2Pose


In [60]:
mp_pose = mp.solutions.pose
pose_model = mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5
)

In [61]:
video_path = r"C:\Users\ivano\Desktop\SBER\ezgif-3-e1fc06a5ab.mp4"
cap = cv2.VideoCapture(video_path)

In [62]:
def calculate_speed(landmarks):
    # Вычислите положение суставов
    shoulder_left = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    shoulder_right = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    hip_left = landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
    hip_right = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]

    # Вычислите смещение суставов
    dx_shoulders = shoulder_right.x - shoulder_left.x
    dy_shoulders = shoulder_right.y - shoulder_left.y
    dx_hips = hip_right.x - hip_left.x
    dy_hips = hip_right.y - hip_left.y

    # Вычислите скорость движения
    speed = (dx_shoulders**2 + dy_shoulders**2 + dx_hips**2 + dy_hips**2)**0.5
    return speed

In [63]:
threshold = 0.43

i = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Преобразуйте кадр в формат RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Выполните обнаружение позы
    results = pose_model.process(frame_rgb)

    # Проверьте, есть ли в кадре человек
    if results.pose_landmarks is not None:
        # Вычислите скорость движения человека
        speed = calculate_speed(results.pose_landmarks)

        # Если скорость движения ниже определенного порога, то это пауза
        if speed < threshold:
            # Сохраните кадр
            cv2.imwrite(f'./cropps/img{i}.jpg' , frame)
            i += 1

In [64]:
shutil.rmtree('./cropps')
os.makedirs('./cropps')